In [2]:
!pip install fastapi uvicorn

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 1.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.1.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
import torch
import torch.nn.functional as F
import numpy as np
import cv2
import io
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import Response
from PIL import Image

# Import your specific model structure
# Ensure the 'Models' folder is in the same directory as this script
from Models.DeepLabV3Plus.modeling import deeplabv3plus_resnet101


In [5]:
NUM_CLASSES = 3
OUTPUT_STRIDE = 8
IMG_SIZE = (512, 512) # Example: Change this to match config.data.img_size
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Loading model on {DEVICE}...")
model = deeplabv3plus_resnet101(
    num_classes=NUM_CLASSES, 
    output_stride=OUTPUT_STRIDE, 
    pretrained_backbone=False # False because we are loading our own weights
)

Loading model on cpu...


In [7]:
checkpoint = torch.load("best.pth", map_location=DEVICE)
# If your checkpoint saves the state dict under a key like 'state_dict' or 'model', adjust below:
# model.load_state_dict(checkpoint['state_dict']) 
model.load_state_dict(checkpoint)

/var/folders/s0/93s512d135s1swchzk_dy_1m0000gn/T/ipykernel_66608/2529535819.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("best.pth", map_locat

<All keys matched successfully>

In [8]:
model.to(DEVICE)
model.eval()
print("Model loaded successfully.")

Model loaded successfully.
